In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [5]:
len(tsla)

1551

In [6]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046112,94.574035,91.246222,91.745864,6840800
2019-01-03,92.009842,93.555908,91.849581,93.329657,6346000
2019-01-04,95.054817,95.752431,93.065670,93.254212,7269100
2019-01-07,97.204208,97.826402,95.460172,95.507303,6263100
2019-01-08,98.278908,98.806831,96.883674,98.212917,9391000


In [3]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [8]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [9]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954697,227.889090,223.117390,223.144604,126925200,NaN,NaN
2019-01-03,221.538956,225.494199,221.049078,225.185752,144140700,NaN,NaN
2019-01-04,228.959625,229.612786,224.224218,224.605226,142628800,NaN,NaN
2019-01-07,230.764877,232.189121,228.324599,229.231765,103139100,NaN,NaN
2019-01-08,232.932968,233.422845,230.420129,232.978343,102512600,NaN,NaN
2019-01-09,234.021591,234.874329,232.406837,233.649650,95006600,NaN,NaN
2019-01-10,234.847137,235.101143,231.780912,232.470367,96823900,NaN,NaN
2019-01-11,234.937836,234.965050,233.168851,233.758503,73858100,NaN,NaN


In [10]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)


                                               title  \
0  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   
1  New Inflation Data Dismays Bulls, CoreWeave IP...   
2  Assessing Apple's Performance Against Competit...   
3  Apple's Options Frenzy: What You Need to Know ...   
4  Trading The SPY As PCE Inflation Report Takes ...   

                                                 url   time_published  \
0  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   
1  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
2  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   
3  https://www.benzinga.com/insights/options/25/0...  20250328T134533   
4  https://www.benzinga.com/markets/equities/25/0...  20250328T125805   

               authors                                            summary  \
0        [Chris Katje]  Benzinga readers pick their favorite Magnifice...   
1   [The Arora Report]  To gain an edge, this is what you need to know

In [11]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,EXCLUSIVE: Which Magnificent 7 Stock Will Perf...,https://www.benzinga.com/tech/25/03/44544129/e...,20250328T190544,[Chris Katje],Benzinga readers pick their favorite Magnifice...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.266216,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1984..."
1,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
2,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
3,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
4,Trading The SPY As PCE Inflation Report Takes ...,https://www.benzinga.com/markets/equities/25/0...,20250328T125805,[RIPS],Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.084859,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0813..."
5,EU To Issue Minimal Fines To Apple And Meta To...,https://www.benzinga.com/markets/25/03/4453457...,20250328T122729,[Namrata Sen],The European Union ( EU ) is reportedly set to...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.103777,Neutral,"[{'ticker': 'META', 'relevance_score': '0.2658..."
6,3 Northern Mutual Funds for Solid Returns,https://www.zacks.com/stock/news/2436613/3-nor...,20250328T111100,[Zacks Equity Research],"Invest in Northern mutual funds like NOIEX, NM...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.272886,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1496..."
7,2 Breakout Growth Stocks You Can Buy and Hold ...,https://www.fool.com/investing/2025/03/28/2-br...,20250328T110700,[Anders Bylund],Looking for undervalued stocks with explosive ...,https://g.foolcdn.com/editorial/images/812691/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.298788,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.1478..."
8,Intel's SuperFluid Cooling Tech 'Possibly Suit...,https://www.benzinga.com/25/03/44532855/intels...,20250328T104410,[Rishabh Mishra],Intel Corp. INTC SuperFluid cooling tech could...,https://editorial-assets.benzinga.com/wp-conte...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.226797,Somewhat-Bullish,"[{'ticker': 'NVDA', 'relevance_score': '0.4078..."
9,Should Strive 500 ETF ( STRV ) Be on Your In...,https://www.zacks.com/stock/news/2436589/shoul...,20250328T102007,[Zacks Equity Research],Style Box ETF report for ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic

## volatility

In [12]:

# Flatten MultiIndex columns 
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate True Range
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#tsla["allocation_dollars"] = tsla["position_size"] * capital

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [13]:
import plotly.express as px
fig = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [14]:
import plotly.express as px

# Filter only 2025 data
tsla_2025 = tsla[tsla.index.year == 2025]

# Plot
fig = px.line(tsla_2025, x=tsla_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [15]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI+_"],
    mode='lines',
    name='VI+_',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI-_"],
    mode='lines',
    name='VI-_',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for TSLA",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()


In [20]:
tsla_2025 = tsla.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI+_"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI-_"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for TSLA - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for SPY",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [19]:
spy_2025 = spy.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for SPY - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for XLY",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [18]:
xly_2025 = xly.loc["2025"]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI+"],
    mode='lines',
    name='VI+',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI-"],
    mode='lines',
    name='VI-',
    line=dict(color='orange')
))

fig.update_layout(
    title="Vortex Indicator (VI+ and VI−) for XLY - 2025",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(x=0, y=1.1, orientation="h"),
    template="plotly_white"
)

fig.show()

In [28]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("SPY - 2025", "XLY - 2025", "TSLA - 2025")
)

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI+"],
    name="VI+ (SPY)",
    line=dict(color='blue'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=spy_2025.index,
    y=spy_2025["VI-"],
    name="VI- (SPY)",
    line=dict(color='orange'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI+"],
    name="VI+ (XLY)",
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=xly_2025.index,
    y=xly_2025["VI-"],
    name="VI- (XLY)",
    line=dict(color='orange'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI+_"],
    name="VI+ (TSLA)",
    line=dict(color='blue'),
    showlegend=False
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=tsla_2025.index,
    y=tsla_2025["VI-_"],
    name="VI- (TSLA)",
    line=dict(color='orange'),
    showlegend=False
), row=3, col=1)

fig.update_layout(
    height=500, width=1200,
    title_text="Vortex Indicator (VI+ and VI−) - 2025 Comparison",
    template="plotly_white"
)

fig.show()

In [27]:
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("SPY Year To Year", "XLY Year To Year", "TSLA Year To Year")
)

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI+"],
    name="VI+ (SPY)",
    line=dict(color='blue'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=spy.index,
    y=spy["VI-"],
    name="VI- (SPY)",
    line=dict(color='orange'),
    showlegend=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI+"],
    name="VI+ (XLY)",
    line=dict(color='blue'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=xly.index,
    y=xly["VI-"],
    name="VI- (XLY)",
    line=dict(color='orange'),
    showlegend=False
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI+_"],
    name="VI+ (TSLA)",
    line=dict(color='blue'),
    showlegend=False
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla["VI-_"],
    name="VI- (TSLA)",
    line=dict(color='orange'),
    showlegend=False
), row=3, col=1)

fig.update_layout(
    height=900, width=1200,
    title_text="Vortex Indicator (VI+ and VI−) - Full Period Comparison",
    template="plotly_white"
)

fig.show()


## Optimization

In [13]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
def calculate_vortex(df, value, n):
    """Calculate Vortex Indicator VI+ and VI-."""
    high = df[("High_"+value)]
    low = df[("Low_"+value)]
    close = df[("Close_"+value)]

    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [16]:
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

In [33]:
import vectorbt as vbt

# Step 3: Backtest Multiple Periods
periods = [7, 14, 21, 30]
results = {}

for n in periods:
    # Compute Vortex Indicator for each period
    tsla[f'VI+_{n}'], tsla[f'VI-_{n}'] = calculate_vortex(tsla, 'TSLA', n)

    # Define Buy and Sell Signals
    tsla[f'Buy_{n}'] = tsla[f'VI+_{n}'] > tsla[f'VI-_{n}']  # Buy when VI+ > VI-
    tsla[f'Sell_{n}'] = tsla[f'VI-_{n}'] > tsla[f'VI+_{n}']  # Sell when VI- > VI+

    # Convert Signals to Boolean Series
    entries = tsla[f'Buy_{n}']
    exits = tsla[f'Sell_{n}']

    # Run Backtest
    portfolio = vbt.Portfolio.from_signals(
        close=tsla[("Close_TSLA")],
        entries=entries,
        exits=exits,
        size=1,  # Buy 1 share per trade
        init_cash=10000
    )
    # Store Performance Metrics
    stats = portfolio.stats()
    results[n] = stats

    print(f"Performance for {n}-day Vortex Indicator:")
    print(stats)
    print("-" * 50)

# results
# Step 4: Plot Best Performing Strategy
best_period = max(results, key=lambda x: results[x]['Total Return [%]'])
print(f"Best Performing Period: {best_period} days")

portfolio = vbt.Portfolio.from_signals(
    close=tsla['Close_TSLA'],
    entries=tsla[f'VI+_{best_period}'] > tsla[f'VI-_{best_period}'],
    exits=tsla[f'VI-_{best_period}'] > tsla[f'VI+_{best_period}'],
    size=1,
    init_cash=10000
)

portfolio.plot().show()


/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requ

Performance for 7-day Vortex Indicator:
Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                               10000.0
End Value                            10480.194603
Total Return [%]                         4.801946
Benchmark Return [%]                  1215.813231
Max Gross Exposure [%]                   4.554966
Total Fees Paid                               0.0
Max Drawdown [%]                         0.793073
Max Drawdown Duration                       351.0
Total Trades                                  113
Total Closed Trades                           113
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            44.247788
Best Trade [%]                         128.434899
Worst Trade [%]                        -15.721837
Avg Winning Trade [%]                   14.052436
Avg Losing